Import dependencies

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

Reading in the excel file. The Export from Raisers Edge should have any fields you want to filter on outside of RE + the Constituent ID, the gift or Gl post date and gift amount.  

In [ ]:
data = pd.read_excel('../Desktop/All Funds Export for BI report.xls')
data.head()

Filters the data and includes in output if any one of the following Conditions is true. Currently commented out.

In [ ]:
alldata_filtered = data
#[
 #   data['Gf_Campaign'].str.contains("Friends")| 
  #  data['Gf_Appeal'].str.contains('Performer')|
  #  data['Gf_Appeal'].str.contains('Playwright')|
  #  data['Gf_Appeal'].str.contains('Director')|
  #  data['Gf_Appeal'].str.contains('Producer')|
  #  data['Gf_Appeal'].str.contains('Presidents')|
  #  data['Gf_Appeal'].str.contains('Chairman')|
  #  data['Gf_Appeal'].str.contains('Performer')|
  #  data['Gf_Appeal'].str.contains('Inspiration')
    
        # ]

#data['Gf_AttrCat_1_01_Description'].notna()

In [ ]:
alldata_filtered.head()

Write the filtered file to excel for review

In [ ]:
#alldata_filtered.to_excel("Filtered_Data.xlsx",sheet_name='Sheet_name_1')  

Remove unnecessary columns

In [ ]:
filtered_for_FundraisingRC = alldata_filtered[['Gf_CnBio_ID','Gf_GL_post_date','Gf_Amount']]

Rename the columns to match the required format for the Fundraising Report Card website

In [ ]:
filtered_for_FundraisingRC = filtered_for_FundraisingRC.rename(columns={'Gf_CnBio_ID': 'donor_id', 'Gf_GL_post_date':'donation_date','Gf_Amount': 'gift_amount'})

In [ ]:
filtered_for_FundraisingRC.to_csv('Upload to FRRC.csv')

In [ ]:
filtered_for_FundraisingRC['Year'] = filtered_for_FundraisingRC['donation_date'].dt.year

In [ ]:
filtered_for_FundraisingRC['Month'] = filtered_for_FundraisingRC['donation_date'].dt.month

In [ ]:
filtered_for_FundraisingRC['As Quarter'] = filtered_for_FundraisingRC['donation_date'].dt.to_period('Q-Jun')

In [ ]:
filtered_for_FundraisingRC['Fiscal Year'] = filtered_for_FundraisingRC['As Quarter'].dt.qyear

In [ ]:
filtered_for_FundraisingRC['Fiscal YearRange'] = filtered_for_FundraisingRC['donation_date'].dt.to_period('Q-Jun').dt.qyear.apply(lambda x: str(x-1) + "-" + str(x))

Number of Donations

In [ ]:
donation_count = filtered_for_FundraisingRC.groupby('Fiscal Year').agg({'donor_id':'size'})
donation_count_df = pd.DataFrame(donation_count)
donation_count_df

Number of Donors

In [ ]:
donor_count = filtered_for_FundraisingRC.groupby('Fiscal Year').agg({'donor_id':'nunique'})
donor_count_df = pd.DataFrame(donor_count)
donor_count_df    


Total Amount donated per Fiscal Year

In [ ]:
sum_donations = filtered_for_FundraisingRC.groupby('Fiscal Year').agg({'gift_amount':'sum'})
sum_donations = pd.DataFrame(sum_donations)
sum_donations['gift_amount']=sum_donations['gift_amount'].map("${:,.2f}".format)
sum_donations

In [ ]:
avg_donations = filtered_for_FundraisingRC.groupby('Fiscal Year').agg({'gift_amount':'mean'})
avg_donations = pd.DataFrame(avg_donations)
avg_donations['avg_gift_amount']=avg_donations['gift_amount'].map("${:,.2f}".format)
avg_donations =avg_donations.drop('gift_amount',1) 
avg_donations

In [ ]:
pivot=filtered_for_FundraisingRC.pivot_table(index=['donor_id'],aggfunc={"donor_id": lambda x: len(x.unique())},  columns='Fiscal Year', values='donation_date')
pivot = pivot.fillna(0)

pivot

In [ ]:
pivot['retained 2021']= ((pivot[2020] == 1) & (pivot[2021] == 1)) # retained 2021


In [ ]:
pivot['retained 2022']= ((pivot[2021] == 1) & (pivot[2022] == 1)) # retained 2022
pivot

In [ ]:
pivot['lapsed 2021']=((pivot[2020] == 1) & (pivot[2021] != 1)) # lapsed 2021

In [ ]:
pivot['lapsed 2022']=((pivot[2021] == 1) & (pivot[2022] != 1))# lapsed 2022

In [ ]:
pivot['New/reengaged 2022']=((pivot[2021] == 0) & (pivot[2022] == 1)) #Reengaged 2022

In [ ]:
pivot

In [ ]:
pivot.to_csv('lapsed_retained.csv')   